# CISM grid creation 
This notebook creates a CISM cartesion grid NetCDF file given the end points of the cartesian dimensions. 

Right now, by default, the grids for the Antarctic and Greenland ice sheets use the same boundaries as those used in the ISMIP6 protocole. 

Author: gunterl@ucar.edu

In [7]:
#Import packages
import numpy as np      
from netCDF4 import Dataset
import sys, os
from pathlib import Path

# to display figures in notebook after executing the code.
%matplotlib inline 

In [9]:
# Configuration

IS = 'GrIS'  # Ice sheet for which we are creating the grid
res = 4000  # grid resolution (m) (CISM has a uniform grid)

precision = 'float'  # Numerical precision for storing the data to NetCDF

# Defining the file to write the CISM grid NetCDF file
pathlocal = os.getcwd()
dstPath = f"{pathlocal}/cism_grid/"

# Creating the path where the Lat-Lon file will be written if it does not exist
Path(dstPath).mkdir(parents=True, exist_ok=True)

In [10]:
if IS not in ['AIS', 'GrIS']:
    sys.exit(f"Chose an ice sheet option in the list ['GrIS', 'AIS']")

str_res = str(res).zfill(5)
dstFile = dstPath + 'xy_CISM3_grid_' + IS + '_' + str_res + '.nc'

if precision in ['float']:
    prec = 'f4'
elif precision in ['double']:
    prec = 'f8'
else:
    sys.exit(f"chose a precision in this list ['float', 'double']")

In [11]:
# Defining the bounds of ISMIP6 domain

if IS in ['AIS']:
    xs = -3040000
    xe = 3040000
    ys = -3040000
    ye = 3040000

if IS in ['GrIS']:
    xs = -720000
    xe = 960000
    ys = -3450000
    ye = -570000

In [12]:
# Defining the grid

nx1 = int((xe-xs)/res + 1)
ny1 = int((ye-ys)/res + 1)
nx0 = nx1 - 1
ny0 = ny1 - 1
print('nx1 =', nx1)
print('ny1 =', ny1)

# Grid for thickness points
x1_dst = np.linspace(xs, xe, nx1)
y1_dst = np.linspace(ys, ye, ny1)
# print('x1 =', x1_dst[-10::])
# print('y1 =', y1_dst[-10::])

# Grid for velocity points
x0_dst = np.linspace(xs+res/2, xe-res/2, nx0)
y0_dst = np.linspace(ys+res/2, ye-res/2, ny0)
# print('x0 =', x0_dst[-10::])
# print('y0 =', y0_dst[-10::])

nx1 = 421
ny1 = 721


In [13]:
# Writing the grid to netCDF file

if os.path.isfile(dstFile):
    os.remove(dstFile)    

ncid = Dataset(dstFile,'w')
ncid.createDimension('x1', nx1)
ncid.createDimension('y1', ny1)
ncid.createDimension('x0', nx0)
ncid.createDimension('y0', ny0)

x1           = ncid.createVariable('x1',prec,('x1',))
x1.long_name = "Cartesian centered thickness x-coordinate"
x1.units     = "m"
x1[:] = x1_dst[:]

y1           = ncid.createVariable('y1',prec,('y1',))
y1.long_name = "Cartesian centered thickness y-coordinate"
y1.units     = "m"
y1[:] = y1_dst[:]

x0           = ncid.createVariable('x0',prec,('x0',))
x0.long_name = "Cartesian x-coordinate, velocity grid"
x0.units     = "m"
x0[:] = x0_dst[:]

y0           = ncid.createVariable('y0',prec,('y0',))
y0.long_name = "Cartesian y-coordinate, velocity grid"
y0.units     = "m"
y0[:] = y0_dst[:]

ncid.close()